In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from datetime import timedelta, datetime

import plotly.graph_objs as go
import plotly.express as ex
from plotly.offline import iplot, init_notebook_mode
import cufflinks
cufflinks.go_offline(connected=True)
init_notebook_mode(connected=True)

# 1. Full building data

## 1.1 Read the data and print the descriptive statistics

In [2]:
def ReadFile(file_name):
    X = pd.read_csv(file_name, na_values = ['-999'])
    
    #Combine the Date and Time column and change it to date time format
    X['Date'] = X['Date'] + " " + X['Time']
    X['Date'] = pd.to_datetime(X['Date'], format = "%d/%m/%Y %H:%M:%S")

    #Delete Time column and change Date column to Datetime
    X.drop(['Time'], inplace = True, axis = 1)
    X.rename(columns = {'Date': 'Datetime'}, inplace = True)
    
    #The testing period start from 16 PM to 13 PM the next day
    #Delate non-testing rows
    mask = (((X['Datetime'].dt.day == X['Datetime'][0].day) & (X['Datetime'].dt.hour >= 16))
           |((X['Datetime'].dt.day == X['Datetime'][len(X) - 1].day) & (X['Datetime'].dt.hour < 13)))
    
    X = X.loc[mask]
    X.reset_index(drop = True, inplace = True)
    X.set_index('Datetime', inplace = True)
    X.dropna(inplace = True)
    return X

In [3]:
no_model = ReadFile('no_model_output.csv') #The default setting
linear   = ReadFile('linear_output.csv')   #The linear regression model
base     = ReadFile('base_output.csv')     #The new sepoint or the setpoint formula

In [4]:
no_model.describe()

,Room Temperature,Room Setpoint,Heating Demand,Heating Power,Cooling Demand,Cooling Power,Current Outside Temperature,Current Solar Power,Supply Air Temp
count,1203.000000,1203.000000,1203.000000,1203.000000,1203.000000,1203.000000,1203.000000,1203.000000,1203.000000
mean,19.702876,18.891937,5.779184,71.102174,65.261869,334.444399,-0.910806,3.154140,19.233838
std,1.454221,1.997909,16.780410,207.032695,39.877036,170.929692,0.659034,9.451509,0.080936
min,18.257061,17.000000,0.000000,0.000000,0.000000,0.000000,-1.800000,0.000000,18.962547
25%,18.380668,17.000000,0.000000,0.000000,27.406944,225.595650,-1.400000,0.000000,19.187940
50%,18.639868,17.000000,0.000000,0.000000,100.000000,425.678345,-1.100000,0.000000,19.244528
75%,21.515099,21.000000,0.000000,0.000000,100.000000,440.284684,-0.600000,0.000000,19.288595
max,22.000645,21.000000,100.000000,1214.930664,100.000000,651.048340,0.800000,40.790001,19.461399


In [5]:
base.describe()

,Room Temperature,Room Setpoint,Heating Demand,Heating Power,Cooling Demand,Cooling Power,Current Outside Temperature,Current Solar Power,Supply Air Temp
count,1221.000000,1221.000000,1221.000000,1221.000000,1221.000000,1221.000000,1221.000000,1221.000000,1221.000000
mean,19.432770,18.893311,14.332069,180.928337,58.170948,270.627640,-0.917281,3.096937,19.237568
std,1.481048,1.933558,34.588134,436.312938,43.790175,180.169105,0.656534,9.415542,0.080987
min,18.061096,17.000000,0.000000,0.000000,0.000000,0.000000,-1.800000,0.000000,18.969099
25%,18.127600,17.000000,0.000000,0.000000,0.000000,0.000000,-1.400000,0.000000,19.188412
50%,18.657190,17.866667,0.000000,0.000000,69.042648,394.052032,-1.100000,0.000000,19.246002
75%,21.497732,21.000000,0.000000,0.000000,100.000000,403.770691,-0.600000,0.000000,19.296282
max,22.031361,21.000000,100.000000,1354.553467,100.000000,631.957031,0.800000,40.790001,19.439291


In [6]:
linear.describe()

,Room Temperature,Room Setpoint,Heating Demand,Heating Power,Cooling Demand,Cooling Power,Current Outside Temperature,Current Solar Power,Supply Air Temp
count,1192.000000,1192.000000,1192.000000,1192.000000,1192.000000,1192.000000,1192.000000,1192.000000,1192.000000
mean,19.444671,18.799639,12.281944,156.707272,62.464426,296.328057,-0.918456,3.120696,19.234312
std,1.452253,1.967237,29.287185,374.816429,42.087755,167.628545,0.660403,9.455721,0.088597
min,18.172733,16.914761,0.000000,0.000000,0.000000,0.000000,-1.800000,0.000000,18.962847
25%,18.190251,16.945565,0.000000,0.000000,22.737809,183.714111,-1.400000,0.000000,19.182907
50%,18.327984,16.990645,0.000000,0.000000,100.000000,404.012909,-1.100000,0.000000,19.242074
75%,21.476887,20.972451,0.000000,0.000000,100.000000,411.537773,-0.600000,0.000000,19.299053
max,21.850519,21.038401,100.000000,1376.194580,100.000000,616.296448,0.800000,40.790001,19.499220


The Room Temperature is in normal condition. The Room Setpoint does not vary much compared to the original one.

## 1.2 Plot the Room Temperature and Room Setpoint

In [7]:
no_model.loc[ : , ['Room Temperature', 'Room Setpoint']].iplot(width = 2, xTitle = 'Date')

In [8]:
base.loc[ : , ['Room Temperature', 'Room Setpoint']].iplot(width = 2, xTitle = 'Date')

In [9]:
linear.loc[ : , ['Room Temperature', 'Room Setpoint']].iplot(width = 2, xTitle = 'Date')

The transition during the Room Setpoint adjustment time is still harsh, different with the base/the labels.

## 1.3 Plot the Heating/Cooling Demand

In [10]:
no_model.loc[ : , ['Heating Demand', 'Cooling Demand']].iplot(width = 2, xTitle = 'Date')

In [11]:
base.loc[ : , ['Heating Demand', 'Cooling Demand']].iplot(width = 2, xTitle = 'Date')

In [12]:
linear.loc[ : , ['Heating Demand', 'Cooling Demand']].iplot(width = 2, xTitle='Date')

The Heating/Cooling Demand are more than the default setting. Thus, the total power consumption might be bigger than the default one.

## 1.4 Calculate the working time of the heating and cooling system

In [13]:
no_model.reset_index(drop = False, inplace = True)
base.reset_index(drop = False, inplace = True)
linear.reset_index(drop = False, inplace = True)

In [14]:
def CalculateTurnOnTime(X):
    #Set the variable
    start = 0
    count = X['Datetime'][0] - X['Datetime'][0]
    
    #Find the first timestamp that Heating/Cooling Demand is not equal 0
    while (X['Heating Demand'][start] == 0 and X['Cooling Demand'][start] == 0):
        start += 1
        
    #Start from there
    i = start
    while (i < len(X)):
        #Now find the next timestamp that Heating/Cooling Deman equals 0
        if (X['Heating Demand'][i] == 0 and X['Cooling Demand'][i] == 0):
            #The time can be calculated by taking the start time - the timestamp before both column equals 0
            count += X['Datetime'][i-1] - X['Datetime'][start]
            #Find the next timestamp that not equal 0 anymore and assign start from there
            while (X['Heating Demand'][i] == 0 and X['Cooling Demand'][i] == 0 and i < len(X)):
                i += 1
            start = i
        else:
            i += 1
    
    #The while loop doesn't account for last period turning time. If in the last row, the
    #Heating/Cooling Demand is not equal 0, we add the time to the count 1 more time
    if (X['Heating Demand'][i-1] != 0 or X['Cooling Demand'][i-1] != 0):
        count += X['Datetime'][i-1] - X['Datetime'][start]
    return count

In [15]:
print("The test runs total for:                     ", no_model['Datetime'][len(no_model) - 1] - no_model['Datetime'][0])
print("The turn on time for default setting:        ", CalculateTurnOnTime(no_model))
print("The turn on time for the setpoint formula:   ", CalculateTurnOnTime(base))
print("The turn on time for linear regression model:", CalculateTurnOnTime(linear))

The test runs total for:                      0 days 20:59:00
The turn on time for default setting:         0 days 19:55:00
The turn on time for the setpoint formula:    0 days 18:16:00
The turn on time for linear regression model: 0 days 20:13:00


The turn on time for the base reduces nearly 1:45 hours, while the linear model one increase more than 15 minutes.

## 1.5 Calculate the violated time with original comfort range

In [16]:
#Create the dataset of upper and lower comfort range
temp = base[['Datetime', 'Room Setpoint']].copy()
mask = ((temp['Datetime'].dt.hour >= 8) & (temp['Datetime'].dt.hour <= 17))
temp['Upper Comfort Range'] = np.where(mask, temp['Room Setpoint'] + 0.5, temp['Room Setpoint'] + 1.0)
temp['Lower Comfort Range'] = np.where(mask, temp['Room Setpoint'] - 0.5, temp['Room Setpoint'] - 1.0)

In [17]:
temp.sample(5)

,Datetime,Room Setpoint,Upper Comfort Range,Lower Comfort Range
742,2020-12-11 04:50:30,17.000000,18.000000,16.000000
594,2020-12-11 02:18:30,17.000000,18.000000,16.000000
925,2020-12-11 07:57:30,18.799999,19.799999,17.799999
699,2020-12-11 04:07:30,17.000000,18.000000,16.000000
1051,2020-12-11 10:03:30,21.000000,21.500000,20.500000


In [18]:
def CalculateViolatedTime(X, temp):
    #Set the variable
    start = 0
    count = X['Datetime'][0] - X['Datetime'][0]
    
    #Find the first timestamp that out of comfort range
    while (X['Room Temperature'][start] <= temp['Upper Comfort Range'][start] and
           X['Room Temperature'][start] >= temp['Lower Comfort Range'][start]):
        start += 1
        
    #Start from there
    i = start
    while (i < len(X)):
        #Now find the next timestamp that out of comfort range
        if (X['Room Temperature'][i] <= temp['Upper Comfort Range'][i] and
            X['Room Temperature'][i] >= temp['Lower Comfort Range'][i]):
            #The time can be calculated by taking the start time - the timestamp before both column equals 0
            count += X['Datetime'][i - 1] - X['Datetime'][start]
            #Find the next timestamp that out of comfort range and assign start from there
            while (X['Room Temperature'][i] <= temp['Upper Comfort Range'][i] and
                   X['Room Temperature'][i] >= temp['Lower Comfort Range'][i] and
                   i < len(X)):
                i += 1
            start = i
        else:
            i += 1
    
    #The while loop doesn't account for last period turning time. If in the last row, the
    #Heating/Cooling Demand is not equal 0, we add the time to the count 1 more time
    if (X['Room Temperature'][i-1] > temp['Upper Comfort Range'][i-1] or
        X['Room Temperature'][i-1] < temp['Lower Comfort Range'][i-1]):
        count += X['Datetime'][i-1] - X['Datetime'][start]
    return count

In [19]:
print("Violated time with the original comfort range          : ", CalculateViolatedTime(no_model, temp))
print("Violated time with the original comfort range (base):    ", CalculateViolatedTime(base, temp))
print("Violated time with the original comfort range (linear):  ", CalculateViolatedTime(linear, temp))

Violated time with the original comfort range          :  0 days 18:39:00
Violated time with the original comfort range (base):     0 days 18:00:00
Violated time with the original comfort range (linear):   0 days 17:31:00


In [20]:
no_model.set_index('Datetime', inplace = True)
base.set_index('Datetime', inplace = True)
linear.set_index('Datetime', inplace = True)

# 2. GUI Data

In [21]:
def ReadFile2(file_name):
    X = pd.read_csv(file_name, delimiter = ';', skiprows = 2, names = ['Datetime', 'Power'])
    X['Datetime'] = pd.to_datetime(X['Datetime'], format = "%Y-%m-%d %H:%M:%S")
    
    mask = (((X['Datetime'].dt.day == X['Datetime'][0].day) & (X['Datetime'].dt.hour >= 16))
           |((X['Datetime'].dt.day == X['Datetime'][len(X) - 1].day) & (X['Datetime'].dt.hour < 13)))
    
    X = X.loc[mask]
    X.reset_index(drop = True, inplace = True)
    
    return X

In [22]:
def ReadFile3(file_name):
    X = pd.read_csv(file_name, names = ['Datetime', 'Power'])
    X['Datetime'] = pd.to_datetime(X['Datetime'], format = "%Y-%m-%dT%H:%M:%S")

    mask = (((X['Datetime'].dt.day == X['Datetime'][0].day) & (X['Datetime'].dt.hour >= 16))
           |((X['Datetime'].dt.day == X['Datetime'][len(no_model) - 1].day) & (X['Datetime'].dt.hour < 13)))
    
    X = X.loc[mask]
    X.reset_index(drop = True, inplace = True)
    
    return X

In [23]:
no_model_cooling = ReadFile2('no_model_cooling_power.csv')
no_model_heating = ReadFile2('no_model_heating_power.csv')
base_cooling     = ReadFile3('base_cooling_power.csv')
base_heating     = ReadFile3('base_heating_power.csv')
linear_cooling   = ReadFile2('linear_cooling_power.csv')
linear_heating   = ReadFile2('linear_heating_power.csv')

In [24]:
print("No model cooling power                = ", round(no_model_cooling['Power'].sum(), 3))
print("No model heating power                =  ",round(no_model_heating['Power'].sum(), 3))
print("No model sum power                    = ", round(no_model_cooling['Power'].sum() + no_model_heating['Power'].sum(), 3))
print()
print("Setpoint formula model cooling power  = ", round(base_cooling['Power'].sum(), 3))
print("Setpoint formula model heating power  = ", round(base_heating['Power'].sum(), 3))
print("Setpoint formula model sum power      = ", round(base_cooling['Power'].sum() + base_heating['Power'].sum(), 3))
print()
print("Linear regression model cooling power = ", round(linear_cooling['Power'].sum(), 3))
print("Linear regression model heating power = ", round(linear_heating['Power'].sum(), 3))
print("Linear regression model sum power     = ", round(linear_cooling['Power'].sum() + linear_heating['Power'].sum(), 3))

No model cooling power                =  421341.95
No model heating power                =   93750.076
No model sum power                    =  515092.026

Setpoint formula model cooling power  =  343659.566
Setpoint formula model heating power  =  227506.021
Setpoint formula model sum power      =  571165.587

Linear regression model cooling power =  371207.82
Linear regression model heating power =  198447.622
Linear regression model sum power     =  569655.442


Like expected, the sum power of the linear regression is much more than the default one. 

Additionally, the base model also consumed more power than the default, even though it turned the heating/cooling system less. This is might be because when heating/cooling was working, it worked on its maximum power, while the default did not always.